In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
import optuna
from sklearn.model_selection import train_test_split
import catboost as cb
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier

import xgboost as xgb

In [2]:
df= pd.read_csv(r"C:\Users\jvrdo\Breast Cancer Lymph Node\ly_pred_clean_3")

In [3]:
df

,Unnamed: 0,Age,Tm_size,??,Hist_type,Gradus,ER,PR,HER-2,Ki-67,Immunohist_type,lymph_meta,Neoadj,Target
0,44,82.0,15.0,difuzno infiltrirajući,lobularni invazivni,2,99.0,95.0,2.0,26.0,luminal B,5,0,1
1,555,52.0,1.2,solitaran,NOS_invazivni,3,0.0,0.0,3.0,60.0,HER2 pozitivni,0,0,0
2,645,62.0,1.6,solitaran,NOS_invazivni,1,100.0,70.0,0.0,7.0,luminal A,0,0,0
3,646,62.0,2.2,solitaran,NOS_invazivni,2,95.0,93.0,1.0,9.0,luminal A,0,0,0
4,647,72.0,2.5,solitaran,lobularni invazivni,2,80.0,60.0,1.0,12.0,luminal A,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,13567,82.0,3.2,solitaran,NOS_invazivni,2,100.0,100.0,0.0,9.0,luminal A,0,1,0
9705,13568,60.0,2.7,solitaran,NOS_invazivni,1,90.0,30.0,2.0,60.0,Luminal B HER2poz,0,1,0
9706,13574,60.0,2.0,multipli,NOS_invazivni,2,100.0,10.0,2.0,18.0,Luminal B HER2poz,3,1,1
9707,13577,42.0,1.1,solitaran,NOS_invazivni,2,98.0,98.0,1.0,65.0,luminal B,1,1,1


In [4]:
df.drop(["lymph_meta","Unnamed: 0"],axis=1,inplace=True)

In [5]:
df

,Age,Tm_size,??,Hist_type,Gradus,ER,PR,HER-2,Ki-67,Immunohist_type,Neoadj,Target
0,82.0,15.0,difuzno infiltrirajući,lobularni invazivni,2,99.0,95.0,2.0,26.0,luminal B,0,1
1,52.0,1.2,solitaran,NOS_invazivni,3,0.0,0.0,3.0,60.0,HER2 pozitivni,0,0
2,62.0,1.6,solitaran,NOS_invazivni,1,100.0,70.0,0.0,7.0,luminal A,0,0
3,62.0,2.2,solitaran,NOS_invazivni,2,95.0,93.0,1.0,9.0,luminal A,0,0
4,72.0,2.5,solitaran,lobularni invazivni,2,80.0,60.0,1.0,12.0,luminal A,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9704,82.0,3.2,solitaran,NOS_invazivni,2,100.0,100.0,0.0,9.0,luminal A,1,0
9705,60.0,2.7,solitaran,NOS_invazivni,1,90.0,30.0,2.0,60.0,Luminal B HER2poz,1,0
9706,60.0,2.0,multipli,NOS_invazivni,2,100.0,10.0,2.0,18.0,Luminal B HER2poz,1,1
9707,42.0,1.1,solitaran,NOS_invazivni,2,98.0,98.0,1.0,65.0,luminal B,1,1


In [6]:
# Creating X and y
X= df.drop(["Target"], axis=1)
y= df["Target"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=68, stratify=y)

In [8]:
# Picking categorical columns
categorical_features_indices = np.where(X_train.dtypes != float)[0]

In [9]:
train_dataset = cb.Pool(data=X_train,
                     label=y_train,
                     cat_features = categorical_features_indices) 
test_dataset = cb.Pool(X_test, y_test,cat_features = categorical_features_indices)

In [18]:
# specifying the cv


def objective(trial):
    param = {
        
        "max_depth":trial.suggest_int("max_depth", 3, 16),
        "n_estimators":trial.suggest_int("n_estimators", 100, 6000),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.2),
        'random_strength':trial.suggest_int("random_strength", 1,10),
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10),
        'max_bin':trial.suggest_categorical('max_bin', [10,200,400]),
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        
        'od_type' : "Iter",
        'od_wait' : 100,
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 1),
         'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,100,500,1024]),
        'custom_metric' : ['AUC'],
        "loss_function": "Logloss",
        
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        }

    scores = cv(train_dataset,
            param,
            fold_count=5, 
            early_stopping_rounds=15,         
            plot=False, verbose=False)

    return scores['test-AUC-mean'].max()

In [19]:
sampler_2 = optuna.samplers.TPESampler(seed=42)  # Make the sampler behave in a deterministic way.
study_2 = optuna.create_study(direction="maximize", sampler=sampler)
study_2.optimize(objective, n_trials=150)

[I 2022-09-20 10:25:21,247] A new study created in memory with name: no-name-b3aa3ef0-19bf-44c7-a080-d22b0de9acdd


Training on fold [0/5]

bestTest = 0.5956235785
bestIteration = 30

Training on fold [1/5]

bestTest = 0.5930638133
bestIteration = 39

Training on fold [2/5]

bestTest = 0.5933493543
bestIteration = 18

Training on fold [3/5]


[I 2022-09-20 10:25:22,160] Trial 0 finished with value: 0.744361535532417 and parameters: {'max_depth': 7, 'n_estimators': 2393, 'min_data_in_leaf': 28, 'learning_rate': 0.16591876432123395, 'random_strength': 4, 'bagging_temperature': 3, 'max_bin': 400, 'grow_policy': 'Depthwise', 'l2_leaf_reg': 3.8879928024075543e-07, 'one_hot_max_size': 10, 'auto_class_weights': 'Balanced'}. Best is trial 0 with value: 0.744361535532417.



bestTest = 0.6128463271
bestIteration = 17

Training on fold [4/5]

bestTest = 0.602489082
bestIteration = 20

Training on fold [0/5]

bestTest = 0.5799077864
bestIteration = 383

Training on fold [1/5]

bestTest = 0.5833580789
bestIteration = 441

Training on fold [2/5]

bestTest = 0.5836462904
bestIteration = 329

Training on fold [3/5]

bestTest = 0.6074162264
bestIteration = 264

Training on fold [4/5]


[I 2022-09-20 10:25:28,053] Trial 1 finished with value: 0.7546568242329302 and parameters: {'max_depth': 15, 'n_estimators': 3778, 'min_data_in_leaf': 34, 'learning_rate': 0.013648111706918705, 'random_strength': 4, 'bagging_temperature': 3, 'max_bin': 400, 'grow_policy': 'Lossguide', 'l2_leaf_reg': 0.012197768563438372, 'one_hot_max_size': 10, 'auto_class_weights': 'Balanced'}. Best is trial 1 with value: 0.7546568242329302.


...


bestTest = 0.5942491201
bestIteration = 119

Training on fold [0/5]

bestTest = 0.5657083251
bestIteration = 241

Training on fold [1/5]

bestTest = 0.5680224202
bestIteration = 252

Training on fold [2/5]

bestTest = 0.5697400846
bestIteration = 170

Training on fold [3/5]

bestTest = 0.5947370077
bestIteration = 148

Training on fold [4/5]


[I 2022-09-20 10:44:21,256] Trial 148 finished with value: 0.7564338301488942 and parameters: {'max_depth': 6, 'n_estimators': 3155, 'min_data_in_leaf': 17, 'learning_rate': 0.03671741542707635, 'random_strength': 4, 'bagging_temperature': 7, 'max_bin': 400, 'grow_policy': 'Depthwise', 'l2_leaf_reg': 0.2889031187794099, 'one_hot_max_size': 1024, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 107 with value: 0.7571597677410438.



bestTest = 0.5823243647
bestIteration = 237

Training on fold [0/5]

bestTest = 0.5769729286
bestIteration = 145

Training on fold [1/5]

bestTest = 0.5804099903
bestIteration = 141

Training on fold [2/5]

bestTest = 0.5816937894
bestIteration = 108

Training on fold [3/5]

bestTest = 0.6049571973
bestIteration = 83

Training on fold [4/5]


[I 2022-09-20 10:44:23,733] Trial 149 finished with value: 0.756155417380221 and parameters: {'max_depth': 7, 'n_estimators': 3286, 'min_data_in_leaf': 33, 'learning_rate': 0.04327758584690818, 'random_strength': 3, 'bagging_temperature': 5, 'max_bin': 400, 'grow_policy': 'Depthwise', 'l2_leaf_reg': 0.6001152302322794, 'one_hot_max_size': 1024, 'auto_class_weights': 'Balanced'}. Best is trial 107 with value: 0.7571597677410438.



bestTest = 0.592948288
bestIteration = 107



In [ ]:
#Trial 107 finished with value: 0.7571597677410438 and parameters: {'max_depth': 5, 'n_estimators': 3747, 'min_data_in_leaf': 41, 
#'learning_rate': 0.016484084665340938, 'random_strength': 2, 'bagging_temperature': 6, 'max_bin': 200, 'grow_policy': 'Depthwise', 
#                   'l2_leaf_reg': 0.13343418537730445, 'one_hot_max_size': 1024, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 107 with value: 0.7571597677410438.

In [69]:
final_model = CatBoostClassifier(verbose=False,  cat_features=categorical_features_indices, 
                          n_estimators=100,
    learning_rate=0.01648408,
    random_strength=2,
    bagging_temperature=6,
    max_bin=200,
    grow_policy="Depthwise",
    min_data_in_leaf=41,
    max_depth=10,
    l2_leaf_reg=0.133434,
    one_hot_max_size=1024,
    auto_class_weights="SqrtBalanced")

In [70]:
final_model.fit(X_train, y_train)

In [71]:
predictions_h1 = final_model.predict_proba(X_test)

In [72]:
predictions_h1

array([[0.7244388 , 0.2755612 ],
       [0.672897  , 0.327103  ],
       [0.50162042, 0.49837958],
       ...,
       [0.75038475, 0.24961525],
       [0.50584793, 0.49415207],
       [0.38701316, 0.61298684]])

In [73]:
predictions_h1 = predictions_h1[:,1].reshape(-1,1)

In [74]:
roc_auc_score(y_test, predictions_h1)

0.750725665390252